In [ ]:
import itertools
from math import *

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from utils.colormap import truncate_colormap
from utils import itertools_recipes

from ml.gradient_descent import GradientDescentOptimizer

In [ ]:
def randomDet1Matrix():
    A = np.random.normal(size=(2, 2))
    A /= sqrt(abs(np.linalg.det(A)))
    return A

reds_m = 80
blues_m = 60

radiuses = np.random.normal(10, 1, size=reds_m) ** 2
angles = np.random.uniform(low=0.0, high=2 * pi, size=reds_m)
A = randomDet1Matrix()  # Random tilt / rotation
b = np.random.normal(0, 40, size=(2, 1)) # Random shift
reds = A @ np.array([radiuses * np.cos(angles), radiuses * np.sin(angles)]) + b
blues = A @ np.random.multivariate_normal([0, 0], [[700, 0], [0, 700]], blues_m).T + b

In [ ]:
plt.scatter(*reds, c='r',  linewidths=0.5, edgecolors='black')
plt.scatter(*blues, c='b',  linewidths=0.5, edgecolors='black');